# Detect an object with OpenCV-Python

source: [link](https://www.geeksforgeeks.org/detect-an-object-with-opencv-python/)

In [2]:
import os 
import cv2 
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
## Conda env (obj-tf)
""" conda create --name obj-tf python=3.9 """ 
""" pip install tensorflow """

In [1]:
import mediapipe as mp

# Object detection task guide
[check link](https://developers.google.com/mediapipe/solutions/vision/object_detector)


In [ ]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

<!-- don't use code below -->

In [ ]:
# Opening image
img = cv2.imread("caputre-images/image-1.jpg")
   
# OpenCV opens images as BRG 
# but we want it as RGB and 
# we also need a grayscale 
# version
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   
# Creates the environment 
# of the picture and shows it
fig, ax = plt.subplots(1, 2, figsize=(10, 20))

# Display RGB on the first subplot
ax[0].imshow(img_rgb)   
ax[0].axis("off")
ax[0].set_title("RGB image")

# Display Grayscale on the second subplot
ax[1].imshow(img_gray , cmap="gray")  
ax[1].axis("off")
ax[1].set_title("Gray image")

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# Load the object detection model from TensorFlow Hub
detector = hub.load("https://www.kaggle.com/models/tensorflow/efficientdet/frameworks/TensorFlow2/variations/d0/versions/1")

# Load the image
img = tf.io.read_file("caputre-images/image-1.jpg")
img = tf.image.decode_jpeg(img)

# Convert the image to a NumPy array
img = np.array(img)

# Convert the image to RGB format (the model expects RGB)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Run the object detection model on the image
results = detector(tf.convert_to_tensor(img_rgb)[tf.newaxis, ...])

# Get the detections (bounding boxes, classes, and scores)
boxes = results[0]["detection_boxes"].numpy()
classes = results[0]["detection_classes"].numpy()
scores = results[0]["detection_scores"].numpy()

# Find the index of the highest scoring detection (assuming you want the most likely object)
highest_score_idx = np.argmax(scores)

# Get the bounding box coordinates of the highest scoring detection
box = boxes[highest_score_idx]

# Convert the bounding box coordinates to pixel values (assuming the image is in BGR format)
(ymin, xmin, ymax, xmax) = box

# Draw a green rectangle around the detected object
cv2.rectangle(img_rgb, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)

# Display the image with the green rectangle
cv2.imshow("Image with Detected Object", img_rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()


KeyboardInterrupt: 